# Water Quality Prediction from Satellite Data
 

# READ CONFIG

In [ ]:
import configparser

# Create parser
config = configparser.ConfigParser()

# Read config file
config.read("config.ini")

# Read PATH variables
IN_SITU_FILE = config["PATHS"]["IN_SITU_FILE"]
SENTINEL_DIR = config["PATHS"]["SENTINEL_DIR"]
LANDSAT_DIR = config["PATHS"]["LANDSAT_DIR"]
OUTPUT_DIR = config["PATHS"]["OUTPUT_DIR"]

# Read list variables and convert to Python lists
SENTINEL_BAND_NAMES = [
    band.strip() 
    for band in config["BANDS"]["SENTINEL_BAND_NAMES"].split(",")
]

LANDSAT_BAND_NAMES = [
    band.strip() 
    for band in config["BANDS"]["LANDSAT_BAND_NAMES"].split(",")
]
PARAMS_COL = [
    p.strip() 
    for p in config["PARAMS"]["PARAMS_COL"].split(",")
]
 

# Print to verify
print("IN_SITU_FILE:", IN_SITU_FILE)
print("SENTINEL_BAND_NAMES:", SENTINEL_BAND_NAMES)

In [ ]:
import geopandas as gpd
import pandas as pd
in_sutu_m = gpd.read_file(IN_SITU_FILE)
in_sutu_m['date']=pd.to_datetime(in_sutu_m['DATUM'])

## Read Sentinel bands

In [ ]:
import os
import re
from datetime import datetime
import pandas as pd
import geopandas as gpd
import rasterio

base_path = SENTINEL_DIR
band_names =  SENTINEL_BAND_NAMES
band_names_s=['S_'+b for b in band_names]
 
folder_date_map = {}
for f in os.listdir(base_path):
    full_path = os.path.join(base_path, f)
    if True:#os.path.isdir(full_path):
        match = re.search(r"(\d{8})", f)
        if match:
            if full_path.endswith('.tif'):
                date_obj = datetime.strptime(match.group(1), "%Y%m%d")
                folder_date_map[date_obj] = full_path

folder_dates = sorted(folder_date_map.keys())

 
gdf=in_sutu_m.copy()
# --- Helper to find nearest available folder date ---
def find_nearest_date(target_date, date_list):
    return min(date_list, key=lambda d: abs(d - target_date))

 

# Prepare empty columns in gdf
for b in band_names_s:
    gdf[b] = None
def find_nearest_date(target_date, date_list):
    return min(date_list, key=lambda d: abs(d - target_date))
for idx, row in gdf.iterrows():
    #print(row['date'],folder_dates)
    nearest_date = find_nearest_date(row['date'], folder_dates)
    folder_path = folder_date_map[nearest_date]
 
    

    point = row.geometry

 
    for tiff_file in [folder_path]:
        # Extract band code from filename (e.g., 'B04')
  
        with rasterio.open( tiff_file) as src:
            if gdf.crs != src.crs:
                        point = gdf.to_crs(src.crs).iloc[idx].geometry
   
            values = list(src.sample([(point.x, point.y)]))[0]
            for i in range(len(values)):
                 gdf.at[idx, band_names_s[i]] = values[i]
                 
 
gdf.head()

## Read Landsat bands

In [ ]:
#import xarray as xr
import numpy as np
import pandas as pd
import os
import re
from datetime import datetime
import geopandas as gpd
import rasterio 
DATA_DIR= LANDSAT_DIR
band_names =  LANDSAT_BAND_NAMES
band_names_l=['L_'+b for b in band_names]

import os
import re
from datetime import datetime

base_path = DATA_DIR
 

folder_date_map = {}
for f in os.listdir(base_path):
    full_path = os.path.join(base_path, f)
    if True:#os.path.isdir(full_path):
        match = re.search(r"(\d{8})", f)
        if match:
            date_obj = datetime.strptime(match.group(1), "%Y%m%d")
            folder_date_map[date_obj] = full_path

folder_dates = sorted(folder_date_map.keys())
 
def find_nearest_date(target_date, date_list):
    return min(date_list, key=lambda d: abs(d - target_date))

 
# Prepare empty columns in gdf
for b in band_names_l:
    gdf[b] = None


def find_nearest_date(target_date, date_list):
    return min(date_list, key=lambda d: abs(d - target_date))
for idx, row in gdf.iterrows():
 
    nearest_date = find_nearest_date(row['date'], folder_dates)
    folder_path = folder_date_map[nearest_date]
    

    point = row.geometry

 
    for tiff_file in [folder_path]:
        
        with rasterio.open( tiff_file) as src:
   
            if gdf.crs != src.crs:
                        point = gdf.to_crs(src.crs).iloc[idx].geometry
         
            values = list(src.sample([(point.x, point.y)]))[0]
            for i in range(len(values)):
                 gdf.at[idx, band_names_l[i]] = values[i]

gdf.head()

In [ ]:
from pathlib import Path
out_dir = Path(OUTPUT_DIR)
out_dir.mkdir(exist_ok=True)
gdf.to_csv(os.path.join(out_dir, 'dataset_sent_land.csv'), index=False)

# Read stored csv

In [ ]:
import pandas as pd
import geopandas as gpd
out_vars=PARAMS_COL
Landsat_bands=['L_'+b for b in LANDSAT_BAND_NAMES]  
Sentinel_bands=['S_'+b for b in SENTINEL_BAND_NAMES]  

# read CSV
df = pd.read_csv(os.path.join(OUTPUT_DIR, 'dataset_sent_land.csv'))
 

## Plot Correlation heatmap

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
fig, axes = plt.subplots(1, 3, figsize=(16, 10))
in_X_cols=Sentinel_bands + Landsat_bands
out_Y_cols=out_vars

corr=df[in_X_cols+ out_Y_cols].corr(method="pearson")
# Correlation of features with each target
 
pearson_corr= corr.loc[in_X_cols,out_Y_cols]

corr2=df[in_X_cols+ out_Y_cols].corr(method="spearman")

spearman_corr= corr2.loc[in_X_cols,out_Y_cols]


corr3=df[in_X_cols+ out_Y_cols].corr(method="kendall")

kendall_corr= corr3.loc[in_X_cols,out_Y_cols]
sns.heatmap(
    pearson_corr,
    annot=True,
    fmt=".2f",
    cmap="coolwarm",
    center=0,
    ax=axes[0]
)
axes[0].set_title("Pearson Correlation")

sns.heatmap(
    spearman_corr,
    annot=True,
    fmt=".2f",
    cmap="coolwarm",
    center=0,
    ax=axes[1]
)
axes[1].set_title("Spearman Correlation")

sns.heatmap(
    kendall_corr,
    annot=True,
    fmt=".2f",
    cmap="coolwarm",
    center=0,
    ax=axes[2]
)
axes[2].set_title("Kendall Correlation")

plt.tight_layout()
plt.show()


#  Define evaluation framework

### Feature transformation

In [ ]:
import numpy as np
bands=Sentinel_bands + Landsat_bands

df=df[df['L_B11']>0]
inv_cols = [f"{c}_inv" for c in bands]
df[inv_cols] = df[bands].where(df[bands] != 0).rdiv(1.0)
df[inv_cols].replace([np.inf, -np.inf], np.nan, inplace=True)

# extend feature list and recompute pearson correlation to include inverses
 
sq_cols = [f"{c}_sq" for c in bands]
num_cols = [c for c in bands if c in df.columns and np.issubdtype(df[c].dtype, np.number)]

df[sq_cols] = df[num_cols].pow(2)
df[sq_cols].replace([np.inf, -np.inf], np.nan, inplace=True)

log_cols = [f"{c}_log" for c in bands]
df[log_cols] = df[num_cols].apply(np.log1p)

df.columns

## Models

In [ ]:
from sklearn.linear_model import (
    LinearRegression, Ridge, Lasso, ElasticNet,
    HuberRegressor, BayesianRidge
)

from sklearn.ensemble import (
    RandomForestRegressor,
    ExtraTreesRegressor,
    GradientBoostingRegressor,
    AdaBoostRegressor,
    HistGradientBoostingRegressor
)

from sklearn.neighbors import KNeighborsRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, Matern

from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

import xgboost as xgb


# ==============================
# MODELS TO BENCHMARK
# ==============================

models1 = {

    # ----- Linear & Regularized -----
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(),
    "Lasso Regression": Lasso(),
    "ElasticNet": ElasticNet(),
    #"Huber Regressor": HuberRegressor(),
    "Bayesian Ridge": BayesianRidge(),

    # ----- Polynomial Regression -----
    "Polynomial Regression": Pipeline([
        ("scaler", StandardScaler()),
        ("poly", PolynomialFeatures(degree=2, include_bias=False)),
        ("linear", LinearRegression())
    ]),

    # ----- Tree-Based Ensembles -----
    "Random Forest": RandomForestRegressor(
        n_estimators=3, random_state=42, n_jobs=-1,max_depth=3
    ),

    "Extra Trees": ExtraTreesRegressor(
        n_estimators=3, random_state=42, n_jobs=-1,max_depth=3
    ),

    "Gradient Boosting": GradientBoostingRegressor(
        n_estimators=300, random_state=42,max_depth=3
    ),

    "HistGradientBoosting": HistGradientBoostingRegressor( ),

    "AdaBoost": AdaBoostRegressor(
        n_estimators=200, random_state=42
    ),

    # ----- XGBoost -----
    "XGBoost": xgb.XGBRegressor(
        n_estimators=300,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1
    ),

    # ----- Kernel Methods -----
    "SVR (RBF)": Pipeline([
        ("scaler", StandardScaler()),
        ("svr", SVR(kernel="rbf", C=10, epsilon=0.1))
    ]),

    "Kernel Ridge (RBF)": KernelRidge(
        kernel="rbf", alpha=1.0, gamma=0.1
    ),

    # ----- Gaussian Process -----
    "Gaussian Process": Pipeline([
    ("scaler", StandardScaler()),
    ("gpr", GaussianProcessRegressor(
        kernel=Matern(nu=1.5),
        normalize_y=True,
        random_state=42
    ))
]),

    # ----- Distance-Based -----
    "KNN Regressor": Pipeline([
        ("scaler", StandardScaler()),
        ("knn", KNeighborsRegressor(n_neighbors=3))
    ]),

    # ----- Neural Network -----
    "ANN (MLP)": MLPRegressor(
        hidden_layer_sizes=(64, 32),
        activation="relu",
        alpha=0.001,
        max_iter=2000,
        random_state=42
    )
}


## Time Splits 

In [ ]:
date_splits = [
    { "splt_kind": "Random Split (baseline)"},
    

    {
        # doktorat / projekt
        "splt_kind": "Strict temporal projection",
        "train": [
            "2023-07-17 00:00:00",
            "2023-08-18 00:00:00",
            "2023-09-27 00:00:00",
            "2023-10-13 00:00:00",
            "2023-12-04 00:00:00",
            "2023-12-19 00:00:00",
            "2024-01-11 00:00:00",
            "2024-02-19 00:00:00",
            "2024-03-14 00:00:00",
            "2024-04-29 00:00:00",
            "2024-05-24 00:00:00",
            "2024-06-17 00:00:00",
            "2024-07-19 00:00:00",
        ],
        "test": [
            "2025-05-28 00:00:00",
            "2025-06-26 00:00:00",
            "2025-07-24 00:00:00",
            "2025-08-27 00:00:00",
            "2025-09-16 00:00:00",
            "2025-10-28 00:00:00",
            "2025-11-12 00:00:00",
            "2025-12-12 00:00:00",
        ],
    },

    {
       
        # dok +3 / projekt -3
        "splt_kind": "Partial forward projection",
        "train": [
            "2023-07-17 00:00:00",
            "2023-08-18 00:00:00",
            "2023-09-27 00:00:00",
            "2023-10-13 00:00:00",
            "2023-12-04 00:00:00",
            "2023-12-19 00:00:00",
            "2024-01-11 00:00:00",
            "2024-02-19 00:00:00",
            "2024-03-14 00:00:00",
            "2024-04-29 00:00:00",
            "2024-05-24 00:00:00",
            "2024-06-17 00:00:00",
            "2024-07-19 00:00:00",
            "2025-05-28 00:00:00",
            "2025-06-26 00:00:00",
            "2025-07-24 00:00:00",
        ],
        "test": [
            "2025-08-27 00:00:00",
            "2025-09-16 00:00:00",
            "2025-10-28 00:00:00",
            "2025-11-12 00:00:00",
            "2025-12-12 00:00:00",
        ],
    },

    {
        # dok +3 last / projekt -3 last
        "splt_kind": "Late season calibration",
        "train": [
            "2023-07-17 00:00:00",
            "2023-08-18 00:00:00",
            "2023-09-27 00:00:00",
            "2023-10-13 00:00:00",
            "2023-12-04 00:00:00",
            "2023-12-19 00:00:00",
            "2024-01-11 00:00:00",
            "2024-02-19 00:00:00",
            "2024-03-14 00:00:00",
            "2024-04-29 00:00:00",
            "2024-05-24 00:00:00",
            "2024-06-17 00:00:00",
            "2024-07-19 00:00:00",
           "2025-05-28 00:00:00",

            "2025-06-26 00:00:00",
            "2025-07-24 00:00:00",
        ],
        "test": [
            "2025-08-27 00:00:00",
            "2025-09-16 00:00:00",
             "2025-10-28 00:00:00",
            "2025-11-12 00:00:00",
            "2025-12-12 00:00:00",
            
        ],
        },

    {
        # dok +3 last / projekt -3 last
        "splt_kind": "Backward projection",
        "train": [
          
            "2023-12-04 00:00:00",
            "2023-12-19 00:00:00",
            "2024-01-11 00:00:00",
            "2024-02-19 00:00:00",
            "2024-03-14 00:00:00",
            "2024-04-29 00:00:00",
            "2024-05-24 00:00:00",
            "2024-06-17 00:00:00",
            "2024-07-19 00:00:00",
            "2025-10-28 00:00:00",
            "2025-11-12 00:00:00",
            "2025-12-12 00:00:00",
             "2025-08-27 00:00:00",
            "2025-09-16 00:00:00",
            "2025-05-28 00:00:00",
            "2025-06-26 00:00:00",
            "2025-07-24 00:00:00",
        ],
        "test": [
           "2023-07-17 00:00:00",
            "2023-08-18 00:00:00",
            "2023-09-27 00:00:00",
            "2023-10-13 00:00:00",
        ],
    },

    {
        # proj + dok -3 / dok -3
        "splt_kind": "Extended calibration",
        "train": [
              "2023-07-17 00:00:00",
            "2023-08-18 00:00:00",
            "2023-09-27 00:00:00",
            "2023-10-13 00:00:00",
            "2023-12-04 00:00:00",
            "2023-12-19 00:00:00",
            "2024-01-11 00:00:00",
            "2024-02-19 00:00:00",
            "2024-03-14 00:00:00",
         
            "2025-05-28 00:00:00",
            "2025-06-26 00:00:00",
            "2025-07-24 00:00:00",
            "2025-08-27 00:00:00",
            "2025-09-16 00:00:00",
            "2025-10-28 00:00:00",
            "2025-11-12 00:00:00",
            "2025-12-12 00:00:00",
        ],
        "test": [
             "2024-04-29 00:00:00",
            "2024-05-24 00:00:00",
            "2024-06-17 00:00:00",
            "2024-07-19 00:00:00",
        ],
    },

]


In [ ]:
from sklearn.base import clone
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
def evaluate_models(models1, X_train, y_train, X_test, y_test, out_Y_cols, splt_kind,feature_names=None):
    n_targets = y_train.shape[1]
    from sklearn.preprocessing import StandardScaler
    print('-------------------------------------------------')
    print('split kind:------',splt_kind)
    print(X_train.shape[0],'/',X_test.shape[0])
    if feature_names is not None:
        print('feature names:------',feature_names)
 

    print(
        f"{'Model':<20} {'Target':<8} "
        f"{'Features':<20} "  
        f"{'MAE_tr':>10} {'RMSE_tr':>10} {'R2_tr':>10} "
        f"{'MAE_te':>10} {'RMSE_te':>10} {'R2_te':>10}"
    )
    print("-" * 110)
    res=[]

    for name, model in models1.items():
        try:
            for i in range(n_targets):

                # IMPORTANT: clone model so each target is independent
                m = clone(model)

                # ----- FIT -----
                m.fit(X_train, y_train[:, i])

                # ----- PREDICT -----
                y_train_pred = m.predict(X_train).reshape(-1, 1)
                y_test_pred  = m.predict(X_test).reshape(-1, 1)

    

                y_train_orig=y_train[:, i]
                y_test_orig=y_test[:, i]
            

                # ----- METRICS -----
                mae_tr  = mean_absolute_error(y_train_orig, y_train_pred)
                rmse_tr = np.sqrt(mean_squared_error(y_train_orig, y_train_pred))
                r2_tr   = r2_score(y_train_orig, y_train_pred)

                mae_te  = mean_absolute_error(y_test_orig, y_test_pred)
                rmse_te = np.sqrt(mean_squared_error(y_test_orig, y_test_pred))
                r2_te   = r2_score(y_test_orig, y_test_pred)

                print(
                    f"{name:<20};  {out_Y_cols[i]:<7};  {splt_kind:<7}; {feature_names:<7};"
                    f"{mae_tr:10.3f}; {rmse_tr:10.3f}; {r2_tr:10.3f}; "
                    f"{mae_te:10.3f}; {rmse_te:10.3f}; {r2_te:10.3f}"
                )
                res.append([name,out_Y_cols[i],splt_kind,feature_names,mae_tr,rmse_tr,r2_tr, mae_te, rmse_te, r2_te])
            

        except Exception as e:
            print(f"{name:<20} ERROR: {e}")
    return res


# Run evaluation

In [ ]:
from sklearn.model_selection import train_test_split
evaluation=[]
out_Y_cols=out_vars
 
for in_X_cols, name in [
    (Sentinel_bands + Landsat_bands, "Original"),
    (  inv_cols, "Inverses"),
    ( sq_cols, "Squared"),
    (  log_cols, "Logarithmic"),
]:
     df1=df.copy()
     df1['date']=pd.to_datetime(df1['DATUM'])
 
      
     df1.dropna(subset=in_X_cols + out_Y_cols, inplace=True)

     for split in date_splits:
        if "train" in split and "test" in split:
 
            train_dates = pd.to_datetime(split["train"])
            test_dates = pd.to_datetime(split["test"])
            train_df = df1[df1["date"].isin(train_dates)]
            test_df  = df1[df1["date"].isin(test_dates)]
            X_train = train_df[in_X_cols].values
            y_train = train_df[out_Y_cols].values 
            X_test = test_df[in_X_cols].values
            y_test = test_df[out_Y_cols].values  
            print (f"\n=== Evaluating with features: {name} ===")

            print(X_train.shape, X_test.shape)
            r=evaluate_models(models1, X_train, y_train, X_test, y_test, out_Y_cols,  split["splt_kind"],  name)
            evaluation.append(r)
        else:
            X = df1[in_X_cols].values
            y = df1[out_Y_cols].values 
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
            print (f"\n=== Evaluating with features: {name} as {split["splt_kind"]}===")
            r=evaluate_models(models1, X_train, y_train, X_test, y_test, out_Y_cols,  split["splt_kind"],  name)
            evaluation.append(r)

eval_df = pd.DataFrame(
    [item for sublist in evaluation for item in sublist],
    columns=[
        "Model", "Target", "Split", "Feature_Set",
        "MAE_train", "RMSE_train", "R2_train",
        "MAE_test", "RMSE_test", "R2_test"
    ]
)


eval_df.to_csv(os.path.join(OUTPUT_DIR,"model_evaluation_results_new.csv"), index=False)
 

## Use only selected, most correlated bands

In [ ]:

xy_dict = {
    "IRB_EC": ['S_B01', 'S_B03', 'S_B04', 'S_B09'],
    "IRB_DO": ['L_B10', 'L_B11'],
    "IRB_TUR": ['S_B02', 'S_B04', 'S_B07', 'S_B12'],
    "IRB_WT": ['L_B10', 'L_B11']
}



for out_Y , in_X_cols in xy_dict.items():
 
    df1=df.copy()
    evaluation=[]
    name=f'Selected for {out_Y}'
  
    out_Y_cols=[out_Y]
    df1.dropna(subset=in_X_cols + out_Y_cols, inplace=True)

    for split in date_splits:
        if "train" in split and "test" in split:

            train_dates = pd.to_datetime(split["train"])
            test_dates = pd.to_datetime(split["test"])

            train_df = df1[df1["date"].isin(train_dates)]
            test_df  = df1[df1["date"].isin(test_dates)]
            X_train = train_df[in_X_cols].values
            y_train = train_df[out_Y_cols].values  # multivariate regression
            X_test = test_df[in_X_cols].values
            y_test = test_df[out_Y_cols].values  # multivariate regression
            print (f"\n=== Evaluating with features: {name} ===")

            print(X_train.shape, X_test.shape)
        else:
            X = df1[in_X_cols].values
            y = df1[out_Y_cols].values 
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
            print (f"\n=== Evaluating with features: {name} as {split["splt_kind"]}===")
          
        r=evaluate_models(models1, X_train, y_train, X_test, y_test, out_Y_cols,  split["splt_kind"],  name)
        evaluation.append(r)
    eval_df = pd.DataFrame(
        [item for sublist in evaluation for item in sublist],
        columns=[
            "Model", "Target", "Split", "Feature_Set",
            "MAE_train", "RMSE_train", "R2_train",
            "MAE_test", "RMSE_test", "R2_test"
        ]
    )

  
    eval_df.to_csv(os.path.join(OUTPUT_DIR,f'model_evaluation_results_new{out_Y_cols[0]}.csv'), index=False)